In [ ]:
# Custom CNN

from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define paths
base_path = '/content/drive/MyDrive/chest_xray'
train_path = os.path.join(base_path, 'train')
test_path = os.path.join(base_path, 'test')

# Define data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    color_mode='grayscale',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    color_mode='grayscale',
    subset='validation'
)

# Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 1)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

model.save(os.path.join(base_path, 'chest_xray_model.keras'))

# Prepare data generator for testing
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    color_mode='grayscale',
    shuffle=False
)


model = load_model(os.path.join(base_path, 'chest_xray_model.keras'))
predictions = model.predict(test_generator, steps=test_generator.samples // test_generator.batch_size)
predicted_classes = (predictions > 0.5).astype(int)

# True labels
true_classes = test_generator.classes
true_classes = true_classes[:len(predicted_classes)]

# Classification report
report = classification_report(true_classes, predicted_classes)
print(report)

# Individual metrics
accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes)
recall = recall_score(true_classes, predicted_classes)
f1 = f1_score(true_classes, predicted_classes)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


Found 4187 images belonging to 2 classes.
Found 1045 images belonging to 2 classes.
Epoch 1/10
130/130 [==============================] - 1070s 8s/step - loss: 0.3200 - accuracy: 0.8578 - val_loss: 0.2335 - val_accuracy: 0.9072
Epoch 2/10
130/130 [==============================] - 48s 368ms/step - loss: 0.1169 - accuracy: 0.9545 - val_loss: 0.1762 - val_accuracy: 0.9346
Epoch 3/10
130/130 [==============================] - 48s 372ms/step - loss: 0.0824 - accuracy: 0.9673 - val_loss: 0.1334 - val_accuracy: 0.9551
Epoch 4/10
130/130 [==============================] - 48s 368ms/step - loss: 0.0751 - accuracy: 0.9702 - val_loss: 0.1272 - val_accuracy: 0.9531
Epoch 5/10
130/130 [==============================] - 47s 365ms/step - loss: 0.0512 - accuracy: 0.9827 - val_loss: 0.1460 - val_accuracy: 0.9580
Epoch 6/10
130/130 [==============================] - 48s 368ms/step - loss: 0.0413 - accuracy: 0.9846 - val_loss: 0.1770 - val_accuracy: 0.9443
Epoch 7/10
130/130 [===========================

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# testing CNN  Model

from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import numpy as np

def predict_pneumonia(model_path, image_path):
    # Load the saved model
    model = load_model(model_path)

    # Load the image file, targeting the size used during training
    img = image.load_img(image_path, target_size=(150, 150), color_mode="grayscale")

    # Convert the image to an array and expand dimensions to fit model input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0  # Rescale image as model expects

    # Make a prediction
    prediction = model.predict(img_array)

    # Output whether pneumonia is detected based on the prediction
    if prediction[0] > 0.5:
        return "Pneumonia detected"
    else:
        return "No pneumonia detected"

x = '/content/drive/MyDrive/chest_xray/chest_xray_model.keras'
y = '/content/drive/MyDrive/chest_xray/test/PNEUMONIA/person100_bacteria_475.jpeg'
result = predict_pneumonia(x, y)
print(result)


1/1 [==============================] - 0s 89ms/step
Pneumonia detected


In [ ]:
from skimage.feature import hog
from skimage.io import imread
from skimage.transform import resize
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
import os
import glob
import joblib  # Import joblib

# Define paths
base_path = '/content/drive/MyDrive/chest_xray'
train_path = os.path.join(base_path, 'train')
test_path = os.path.join(base_path, 'test')
model_path = os.path.join(base_path, 'model')  # Path to save the model

if not os.path.exists(model_path):
    os.makedirs(model_path)

def extract_hog_features_from_directory(directory, label):
    features = []
    labels = []
    image_files = glob.glob(os.path.join(directory, '*.jpeg'))
    for image_file in image_files:
        image = imread(image_file, as_gray=True)
        image_resized = resize(image, (150, 150))
        hog_feature = hog(image_resized, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), block_norm='L2-Hys')
        features.append(hog_feature)
        labels.append(label)
    return features, labels

# Extract features and labels from the training images
features_normal, labels_normal = extract_hog_features_from_directory(os.path.join(train_path, 'NORMAL'), 0)
features_pneumonia, labels_pneumonia = extract_hog_features_from_directory(os.path.join(train_path, 'PNEUMONIA'), 1)

X_train = np.array(features_normal + features_pneumonia)
y_train = np.array(labels_normal + labels_pneumonia)

# Extract features and labels from the testing images
features_normal_test, labels_normal_test = extract_hog_features_from_directory(os.path.join(test_path, 'NORMAL'), 0)
features_pneumonia_test, labels_pneumonia_test = extract_hog_features_from_directory(os.path.join(test_path, 'PNEUMONIA'), 1)

X_test = np.array(features_normal_test + features_pneumonia_test)
y_test = np.array(labels_normal_test + labels_pneumonia_test)

# Create an SVM classifier
clf = SVC(kernel='linear')

# Train the SVM classifier
clf.fit(X_train, y_train)

# Save the trained model
model_filename = os.path.join(model_path, 'svm_chest_xray_classifier.joblib')
joblib.dump(clf, model_filename)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Print out the classification report
print(classification_report(y_test, y_pred))

# Print out the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')



              precision    recall  f1-score   support

           0       0.97      0.41      0.58       234
           1       0.74      0.99      0.85       390

    accuracy                           0.77       624
   macro avg       0.85      0.70      0.71       624
weighted avg       0.82      0.77      0.74       624

Accuracy: 0.7740384615384616


In [ ]:
# Testing of SVM model
from skimage.feature import hog
from skimage.io import imread
from skimage.transform import resize
import joblib
import numpy as np

def predict_pneumonia(image_path, model_path):
    """
    Predicts whether a given chest X-ray image has pneumonia or not.

    Parameters:
    - image_path: Path to the chest X-ray image to be tested.
    - model_path: Path to the saved SVM model.

    Returns:
    - A string indicating whether the image is predicted to have pneumonia or not.
    """
    # Load the saved model
    clf = joblib.load(model_path)

    # Read image in grayscale
    image = imread(image_path, as_gray=True)
    # Resize image to 150x150
    image_resized = resize(image, (150, 150))
    # Extract HOG features
    hog_feature = hog(image_resized, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), block_norm='L2-Hys')
    # Reshape the features to match the input format expected by the classifier
    hog_feature_reshaped = np.array(hog_feature).reshape(1, -1)
    # Predict using the SVM model
    prediction = clf.predict(hog_feature_reshaped)

    # Return the result
    if prediction == 0:
        return "The image is predicted to be NORMAL."
    else:
        return "The image is predicted to have PNEUMONIA."

# Example usage
model_path = '/content/drive/MyDrive/chest_xray/model/svm_chest_xray_classifier.joblib'
image_path = '/content/drive/MyDrive/chest_xray/test/NORMAL/IM-0013-0001.jpeg'  # Update this path to your specific image
result = predict_pneumonia(image_path, model_path)
print(result)


The image is predicted to be NORMAL.


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import timm
from sklearn.metrics import classification_report

# Define paths
base_path = '/content/drive/MyDrive/chest_xray'
train_path = os.path.join(base_path, 'train')
test_path = os.path.join(base_path, 'test')

# Data augmentation
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load datasets
train_dataset = datasets.ImageFolder(root=train_path, transform=train_transforms)
test_dataset = datasets.ImageFolder(root=test_path, transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Initialize the model
model = timm.create_model('vit_base_patch16_224', pretrained=True)
model.head = nn.Linear(model.head.in_features, 2)  # Change the classifier head

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)

# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Fine-tuning strategy: Start with a frozen network and unfreeze gradually
for param in model.parameters():
    param.requires_grad = False
for param in model.head.parameters():
    param.requires_grad = True

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)

    scheduler.step()
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

    # Unfreezing strategy: e.g., unfreeze the last block after 10 epochs
    if epoch == 9:
        for name, param in model.named_parameters():
            if name.startswith('blocks.11'):  # Example: Unfreeze the last block
                param.requires_grad = True

# Evaluation
all_targets = []
all_predictions = []

model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        all_targets.extend(labels.cpu().numpy())
        all_predictions.extend(predictions.cpu().numpy())

print(classification_report(all_targets, all_predictions, target_names=['NORMAL', 'PNEUMONIA']))

# Save the model
model_path = os.path.join(base_path, 'vit_model_finetuned.pth')
torch.save(model.state_dict(), model_path)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Epoch 1/20, Loss: 0.3854
Epoch 2/20, Loss: 0.3039
Epoch 3/20, Loss: 0.2727
Epoch 4/20, Loss: 0.2554


KeyboardInterrupt: 

In [ ]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 944.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━